In [25]:
import pandas as pd
import numpy as np
import pickle
from tqdm.notebook import tqdm

# model
from sklearn.cross_decomposition import PLSRegression

# math
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from math import sqrt, isnan
from sklearn.metrics import r2_score

fp = "G:\\My Drive\\Darby Work\\Ytsma and Dyar 2021 (LOD paper)\\"

#### Compositions

In [2]:
# generate comps
comps_path = fp + "tables\\TableS1_sample_compositions.xlsx"
lanl_comps = pd.read_excel(comps_path, sheet_name = "LANL")
mhc_comps = pd.read_excel(comps_path, sheet_name = "MHC")
comps = pd.merge(mhc_comps, lanl_comps, how = "outer") # merge comps
comps.columns = comps.columns.map(lambda x: x.split()[0])
comps = comps.drop_duplicates(subset = 'Sample') # remove duplicates
comps['Sample'] = comps['Sample'].astype(str)
comps = comps.sort_values(by='Sample')
comps = comps.replace(np.nan, "", regex=True)
cols = comps.columns.drop('Sample')
comps[cols] = comps[cols].apply(pd.to_numeric) # make columns numeric
# add random number assignment
rd = pd.read_excel('Z:\\Millennium Set\\Millennium_COMPS_viewonly.xlsx', usecols=[0,2])
rd = rd.drop([0,1]).rename(columns={'DO NOT TOUCH THIS':'Sample',
                                    'X.1':'rand_num'}).reset_index(drop=True)
comps = pd.merge(rd, comps, how='right', on='Sample')

#### Datasets (baseline removal and normalization already applied)

In [3]:
cl_earth = pd.read_csv(fp+'CL_all_Earth_spectra.csv')
cl_mars = pd.read_csv(fp+'CL_all_Mars_spectra.csv')
cl_vac = pd.read_csv(fp+'CL_all_Vacuum_spectra.csv')
cc_mars = pd.read_csv(fp+'CC_all_Mars_spectra.csv')

#### Sensitivities

In [4]:
sensitivities = pd.read_csv(fp+'instrument_sensitivities.csv')

#### Split test and train

In [5]:
train_250_1000 = comps[comps.rand_num >= 250].reset_index(drop=True)
train_0_750 = comps[comps.rand_num <= 750].reset_index(drop=True)
test_250_1000 = comps[comps.rand_num < 250].reset_index(drop=True)
test_0_750 = comps[comps.rand_num > 750].reset_index(drop=True)

#### Outlier limits
Calculated by 1.5*IQR + Q3 on entire MHC dataset or highest natural sample for doped elements

In [6]:
outlier_limits = pd.read_csv('Z:\\Millennium Set\\NEW_OUTLIER_LIMITS.csv')
iqr_outliers = dict(zip(outlier_limits.element, outlier_limits.iqr_q3_outlier_limit))
dope_outliers = dict(zip(outlier_limits.element, outlier_limits.highest_natural_for_doped))

#### Make models per element

In [7]:
elements = ['MnO', 'Na2O', 'SiO2', 'Li', 'Ni', 'Pb', 'Rb', 'Sr', 'Zn']
n_ranges = ['0-750', '250-1000']
factors = {
'LOB' : 1.645,
'LOD' : 3.3,
'LOQ' : 10
}
methods = ['braga', 'metals']
dfs = [cl_earth,cl_mars,cl_vac,cc_mars]
df_names = ['CL_Earth', 'CL_Mars', 'CL_Vac', 'CC_Mars']
mhc_list = [cl_earth,cl_mars,cl_vac]
outliers = [iqr_outliers, dope_outliers]

In [ ]:
# PLS parameters
n_folds = 5
max_components = 30

# prep for results
n_range_list = []
element_list = []
atm_list = []
inst_list = []
n_train_list = []
rmsecv_list = []
component_list = []
rmsec_list = []
train_r2_list = []
train_adj_r2_list = []
lob_list = []
lod_list = []
loq_list = []
outlier_list = []
n_test_list = []
rmsep_list = []
test_r2_list = []
test_adj_r2_list = []
method_list = []

for n_range in tqdm(n_ranges, desc='Number ranges'):

    if n_range == '0-750':
        all_train = train_0_750
        all_test = test_0_750
    else:
        all_train = train_250_1000
        all_test = test_250_1000


    for element in tqdm(elements, leave=False, desc='Elements'):
        count = 0

        for df in tqdm(dfs, leave=False, desc='Dataset'):
            
            if df_names[count].split('_')[0]=='CC':
                inst='LANL'
            else:
                inst='ChemLIBS'
                
            if df_names[count].split('_')[1]=='Vac':
                atm = 'Vacuum'
            else:
                atm = df_names[count].split('_')[1]

            outpath = "{}\\python_models\\{}_{}\\".format(fp, df_names[count], n_range)
            
            count +=1
            
            count1 = 0
            
            for outlier in outliers:
                
                if count1 == 0:
                    o = 'iqr_q3'
                else:
                    o = 'highest_natural'
                
                count1 += 1
                
                out_lim = outlier[element]
                
                if isnan(out_lim):
                    temp_train = all_train.dropna(subset=[element]).reset_index(drop=True)[['Sample', element]]
                    temp_test = all_test.dropna(subset=[element]).reset_index(drop=True)[['Sample', element]]
                else:
                    temp_train = all_train[all_train[element] <= out_lim].reset_index(drop=True)[['Sample', element]]
                    temp_test = all_test[all_test[element] <= out_lim].reset_index(drop=True)[['Sample', element]]
                    
                # train metadata
                train_names = sorted(set(temp_train.Sample).intersection(df.columns)) # sorted
                y_train = temp_train[temp_train.Sample.isin(train_names)][element].values # already alphabetized
                
                if n_train < n_folds:
                    n_folds = n_train

                # train spectra
                X_train = df[train_names]
                spec_list = []
                for column in X_train.columns:
                    spectrum = list(X_train[column])
                    spec_list.append(spectrum)
                X_train = np.array(spec_list)

#---------------------CROSS--VALIDATION and TRAINING--------------------------#
                cv_dict = {}

                for n_components in np.arange(start=2, stop=max_components+1, step=1):
                    # define model
                    temp_pls = PLSRegression(n_components = n_components, scale=False)
                    # run CV and get RMSE
                    temp_rmsecv = (-cross_val_score(
                        temp_pls, X_train, y_train, cv=n_folds, scoring='neg_root_mean_squared_error'
                    )).mean()
                    # add results to dictionary
                    cv_dict.update({temp_rmsecv : n_components})

                # select parameters of model with lowest rmsecv
                rmsecv = min(list(cv_dict.keys()))
                component = cv_dict[rmsecv]
                model = PLSRegression(n_components = component, scale=False)

                model.fit(X_train, y_train)
                pickle.dump(model, open(outpath+element+'_'+o+'_model.asc', 'wb'), protocol=0)

                coeff = pd.DataFrame(model.coef_)
                coeff.to_csv(outpath+element+'_'+o+'_coeffs.csv', index=False)

                for method in methods:

#---------------------CALCULATE---LBDQ--------------------------#
                    sensitivity = sensitivities[
                        (sensitivities.instrument == inst) &
                        (sensitivities.atmosphere == atm) &
                        (sensitivities.method == method)
                    ]['sensitivity'].iloc[0]

                    # calculate regression vector
                    vector = pow(coeff, 2).sum().pow(.5)  #square root of sum of squares

                    # calculate values
                    lob = factors['LOB'] * sensitivity * vector[0]
                    lod = factors['LOD'] * sensitivity * vector[0]
                    loq = factors['LOQ'] * sensitivity * vector[0]

#---------------------CALIBRATION--ERROR--------------------------#                    
                    train_pred = model.predict(X_train)
                    train_pred_true = pd.DataFrame({
                        'sample' : train_names,
                        'actual' : y_train.flatten().tolist(),
                        'pred' : train_pred.flatten().tolist()
                    })
                    
                    temp = train_pred_true[(train_pred_true.pred < 100) &
                                           (train_pred_true.pred > loq)].copy(deep=True)
                    n_train = len(temp)

                    rmsec = sqrt(mean_squared_error(temp.actual, temp.pred))
                    train_r2 = model.score(X_train,y_train)
                    train_adj_r2 = 1 - (1-train_r2)*(len(temp) - 1) / (len(temp) - (temp.shape[1] - 1) - 1)

                    # fill with <LOQ / >100 wt%
                    loq_df = train_pred_true[train_pred_true.pred < loq].copy(deep=True)
                    loq_df['pred'] = '<LOQ'
                    
                    over_df = train_pred_true[train_pred_true.pred > 100].copy(deep=True)
                    if len(over_df) > 0:
                        over_df['pred'] = '>100 wt%'
                        train_pred_true = pd.concat([temp, over_df, loq_df], ignore_index=True)
                    else:
                        train_pred_true = pd.concat([temp, loq_df], ignore_index=True)
                    train_pred_true.to_csv(outpath+element+"_"+o+"_"+method+'_train_preds.csv', index=False)

#------------------------TEST--MODEL------------------------#
                    # test metadata
                    test_names = sorted(set(temp_test.Sample).intersection(df.columns)) # sorted
                    y_test = temp_test[temp_test.Sample.isin(test_names)][element].values # already alphabetized
                
                    # test spectra
                    X_test = df[test_names]
                    spec_list = []
                    for column in X_test.columns:
                        spectrum = list(X_test[column])
                        spec_list.append(spectrum)
                    X_test = np.array(spec_list)

                    # run predictions
                    test_pred = model.predict(X_test)

                    # get RMSE-P
                    test_pred_true = pd.DataFrame({
                        'sample' : test_names,
                        'actual' : y_test,
                        'pred' : test_pred.flatten().tolist()
                    })
                    
                    temp = test_pred_true[(test_pred_true.pred < 100) &
                                          (test_pred_true.pred > loq)].copy(deep=True)
                    n_test = len(temp)

                    if n_test < 2:
                        test_r2 = 'NA'
                        test_adj_r2 = 'NA'
                        rmsep = 'NA'
                    else:
                        # get RMSE-P
                        rmsep = sqrt(mean_squared_error(temp.actual, temp.pred))
                        # get R2
                        test_r2 = r2_score(temp.actual,temp.pred)
                        # adjusted r2
                        test_adj_r2 = 1 - (1-test_r2)*(len(temp) - 1) / (len(temp) - (temp.shape[1] - 1) - 1)

                    # fill with <LOQ / >100 wt%
                    loq_df = test_pred_true[test_pred_true.pred < loq].copy(deep=True)
                    loq_df['pred'] = '<LOQ'
                    
                    over_df = test_pred_true[test_pred_true.pred > 100].copy(deep=True)
                    if len(over_df) > 0:
                        over_df['pred'] = '>100 wt%'
                        test_pred_true = pd.concat([temp, over_df, loq_df], ignore_index=True)
                    else:
                        test_pred_true = pd.concat([temp, loq_df], ignore_index=True)
                        
                    test_pred_true.to_csv(outpath+element+"_"+o+"_"+method+'_test_preds.csv', index=False)
                                 
                    n_range_list.append(n_range)
                    outlier_list.append(o)
                    method_list.append(method)
                    element_list.append(element)
                    atm_list.append(atm)
                    inst_list.append(inst)
                    n_train_list.append(n_train)
                    rmsecv_list.append(rmsecv)
                    component_list.append(component)
                    lob_list.append(lob)
                    lod_list.append(lod)
                    loq_list.append(loq)
                    rmsec_list.append(rmsec)
                    train_r2_list.append(train_r2)
                    train_adj_r2_list.append(train_adj_r2)
                    n_test_list.append(n_test)
                    rmsep_list.append(rmsep)
                    test_r2_list.append(test_r2)
                    test_adj_r2_list.append(test_adj_r2)

Number ranges:   0%|          | 0/2 [00:00<?, ?it/s]

Elements:   0%|          | 0/9 [00:00<?, ?it/s]

Dataset:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
results = pd.DataFrame({
    'element':element_list,
    'outlier_defn':outlier_list,
    'instrument':inst_list,
    'atmosphere':atm_list,
    'method':method_list,
    'num_range':n_range_list,
    'n_train':n_train_list,
    'rmsecv':rmsecv_list,
    'components':component_list,
    'lob':lob_list,
    'lod':lod_list,
    'loq':loq_list,
    'rmsec':rmsec_list,
    'train_r2':train_r2_list,
    'train_adj_r2':train_adj_r2_list,
    'n_test':n_test_list,
    'rmsep':rmsep_list,
    'test_r2':test_r2_list,
    'test_adj_r2':test_adj_r2_list
})

results.to_csv(fp+'results_011422.csv', index=False)